## Olfactory Nupic Spatial Pooling with Variable Learning Rates

In [1]:
import csv
import datetime
import numpy 
import os
import yaml
import seaborn as sns
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import matplotlib as plt


from nupic.algorithms.sdr_classifier_factory import SDRClassifierFactory
from nupic.algorithms.spatial_pooler import SpatialPooler
from nupic.algorithms.temporal_memory import TemporalMemory
#import date encoder
from nupic.encoders.date import DateEncoder
#import RandomDistributedScalarEncoder
from nupic.encoders.random_distributed_scalar import \
  RandomDistributedScalarEncoder
#import normal ScalarEncoder
from nupic.encoders import ScalarEncoder
#category encoder
from nupic.encoders.category import CategoryEncoder

## Model Params

In [2]:
# Model Params!
MODEL_PARAMS = {
    # Type of model that the rest of these parameters apply to.
    'model': "Classification",

    # Version that specifies the format of the config.
    'version': 1,

    # Intermediate variables used to compute fields in modelParams and also
    # referenced from the control section.
    'aggregationInfo': {   'days': 0,
        'fields': [('chem1', 'sum')],
        'hours': 1,
        'microseconds': 0,
        'milliseconds': 0,
        'minutes': 0,
        'months': 0,
        'seconds': 0,
        'weeks': 0,
        'years': 0},

    'predictAheadTime': None,

    # Model parameter dictionary.
    'modelParams': {
        # The type of inference that this model will perform
        'inferenceType': 'TemporalAnomaly',

        'sensorParams': {
            # Sensor diagnostic output verbosity control;
            # if > 0: sensor region will print out on screen what it's sensing
            # at each step 0: silent; >=1: some info; >=2: more info;
            # >=3: even more info (see compute() in py/regions/RecordSensor.py)
            'verbosity' : 0,

            # Include the encoders we use
            'encoders': {
                u'timestamp_timeOfDay': {
                    'fieldname': u'timestamp',
                    'name': u'timestamp_timeOfDay',
                    'timeOfDay': (21, 0.5),
                    'type': 'DateEncoder'},
                u'timestamp_dayOfWeek': None,
                u'timestamp_weekend': { 
                    'fieldname': u'timestamp',
                    'name': u' timestamp_weekend',
                    'weekend': 21,
                    'type': 'DateEncoder'}, 
                u'chem1':{
                    'clipInput': True,
                    'fieldname': u'chem1',
                    'maxval': 100.0,
                    'minval': 0.0,
                    'n': 50,
                    'name': u'c1',
                    'type': 'ScalarEncoder',
                    'w': 21,
                    'resolution':0.88},
                u'chem1':{
                    'clipInput': True,
                    'fieldname': u'chem1',
                    'maxval': 100.0,
                    'minval': 0.0,
                    'n': 50,
                    'name': u'c1',
                    'type': 'ScalarEncoder',
                    'w': 21,
                    'resolution':0.88},
                u'chem2':{
                    'clipInput': True,
                    'fieldname': u'chem2',
                    'maxval': 100.0,
                    'minval': 0.0,
                    'n': 50,
                    'name': u'c1',
                    'type': 'ScalarEncoder',
                    'w': 21,
                    'resolution':0.88},
                u'chem3':{
                    'clipInput': True,
                    'fieldname': u'chem3',
                    'maxval': 100.0,
                    'minval': 0.0,
                    'n': 50,
                    'name': u'c1',
                    'type': 'ScalarEncoder',
                    'w': 21,
                    'resolution':0.88},
                u'chem4':{
                    'clipInput': True,
                    'fieldname': u'chem4',
                    'maxval': 100.0,
                    'minval': 0.0,
                    'n': 50,
                    'name': u'c1',
                    'type': 'ScalarEncoder',
                    'w': 21,
                    'resolution':0.88},
                u'scent':{'fieldname': u'scent',
                    'categoryList':['lemon','flower','dog','sugar'],
                    'forced':False,
                    'name': u'chem',
                    'type': 'CategoryEncoder',
                    'w': 21,
                    'resolution':0.88},},
            # CategoryEncoder(self, w, categoryList, name='category', verbosity=0, forced=False)
            # A dictionary specifying the period for automatically-generated
            # resets from a RecordSensor;
            #
            # None = disable automatically-generated resets (also disabled if
            # all of the specified values evaluate to 0).
            # Valid keys is the desired combination of the following:
            #   days, hours, minutes, seconds, milliseconds, microseconds, weeks
            #
            # Example for 1.5 days: sensorAutoReset = dict(days=1,hours=12),
            #
            # (value generated from SENSOR_AUTO_RESET)
            'sensorAutoReset' : None,
        },

        'spEnable': True,

        'spParams': {
            # SP diagnostic output verbosity control;
            # 0: silent; >=1: some info; >=2: more info;
            'spVerbosity' : 0,

            # Spatial Pooler implementation selector, see getSPClass
            # in py/regions/SPRegion.py for details
            # 'py' (default), 'cpp' (speed optimized, new)
            'spatialImp' : 'cpp',

            'globalInhibition': 1,

            # Number of cell columns in the cortical region (same number for
            # SP and TM)
            # (see also tpNCellsPerCol)
            'columnCount': 2650,

            'inputWidth': 0,

            # SP inhibition control (absolute value);
            # Maximum number of active columns in the SP region's output (when
            # there are more, the weaker ones are suppressed)
            'numActiveColumnsPerInhArea': 40,

            'seed': 1956,

            # potentialPct
            # What percent of the columns's receptive field is available
            # for potential synapses. At initialization time, we will
            # choose potentialPct * (2*potentialRadius+1)^2
            'potentialPct': 0.5,
            'localAreaDensity': -1.0,
            'boostStrength': 0.0,

            # The default connected threshold. Any synapse whose
            # permanence value is above the connected threshold is
            # a "connected synapse", meaning it can contribute to the
            # cell's firing. Typical value is 0.10. Cells whose activity
            # level before inhibition falls below minDutyCycleBeforeInh
            # will have their own internal synPermConnectedCell
            # threshold set below this default value.
            # (This concept applies to both SP and TM and so 'cells'
            # is correct here as opposed to 'columns')
            'synPermConnected': 0.1,

            'synPermActiveInc': 0.1,

            'synPermInactiveDec': 0.005,
        },

        # Controls whether TM is enabled or disabled;
        # TM is necessary for making temporal predictions, such as predicting
        # the next inputs.  Without TP, the model is only capable of
        # reconstructing missing sensor inputs (via SP).
        'tmEnable' : False,

        'tmParams': {
            # TM diagnostic output verbosity control;
            # 0: silent; [1..6]: increasing levels of verbosity
            # (see verbosity in nupic/trunk/py/nupic/research/TP.py and BacktrackingTMCPP.py)
            'verbosity': 0,

            # Number of cell columns in the cortical region (same number for
            # SP and TM)
            # (see also tpNCellsPerCol)
            'columnCount': 2650,

            # The number of cells (i.e., states), allocated per column.
            'cellsPerColumn': 32,

            'inputWidth': 2048,

            'seed': 1960,

            # Temporal Pooler implementation selector (see _getTPClass in
            # CLARegion.py).
            'temporalImp': 'cpp',

            # New Synapse formation count
            # NOTE: If None, use spNumActivePerInhArea
            #
            # TODO: need better explanation
            'newSynapseCount': 20,

            # Maximum number of synapses per segment
            #  > 0 for fixed-size CLA
            # -1 for non-fixed-size CLA
            #
            # TODO: for Ron: once the appropriate value is placed in TP
            # constructor, see if we should eliminate this parameter from
            # description.py.
            'maxSynapsesPerSegment': 32,

            # Maximum number of segments per cell
            #  > 0 for fixed-size CLA
            # -1 for non-fixed-size CLA
            #
            # TODO: for Ron: once the appropriate value is placed in TP
            # constructor, see if we should eliminate this parameter from
            # description.py.
            'maxSegmentsPerCell': 128,

            # Initial Permanence
            # TODO: need better explanation
            'initialPerm': 0.21,

            # Permanence Increment
            'permanenceInc': 0.1,

            # Permanence Decrement
            # If set to None, will automatically default to tpPermanenceInc
            # value.
            'permanenceDec' : 0.1,

            'globalDecay': 0.0,

            'maxAge': 0,

            # Minimum number of active synapses for a segment to be considered
            # during search for the best-matching segments.
            # None=use default
            # Replaces: tpMinThreshold
            'minThreshold': 9,

            # Segment activation threshold.
            # A segment is active if it has >= tpSegmentActivationThreshold
            # connected synapses that are active due to infActiveState
            # None=use default
            # Replaces: tpActivationThreshold
            'activationThreshold': 12,

            'outputType': 'normal',

            # "Pay Attention Mode" length. This tells the TM how many new
            # elements to append to the end of a learned sequence at a time.
            # Smaller values are better for datasets with short sequences,
            # higher values are better for datasets with long sequences.
            'pamLength': 1,
        },

        'clParams': {
            'regionName' : 'SDRClassifierRegion',

            # Classifier diagnostic output verbosity control;
            # 0: silent; [1..6]: increasing levels of verbosity
            'verbosity' : 0,

            # This controls how fast the classifier learns/forgets. Higher values
            # make it adapt faster and forget older patterns faster.
            'alpha': 0.005,

            # This is set after the call to updateConfigFromSubConfig and is
            # computed from the aggregationInfo and predictAheadTime.
            'steps': '1',

            'implementation': 'cpp',
        },

        'anomalyParams': {
            u'anomalyCacheRecords': None,
            u'autoDetectThreshold': None,
            u'autoDetectWaitRecords': 2184
        },

        'trainSPNetOnlyIfRequested': False,
    },
}

## Encoding Data

In [3]:
_NUM_RECORDS = 16000
_EXAMPLE_DIR = os.path.dirname(os.path.abspath('/Users/adamcasper/nupic/examples'))
_INPUT_FILE_PATH = "data50_with_dopa.csv"

In [4]:
#feeding model params to different encoders
modelParams = MODEL_PARAMS["modelParams"]
enParams = modelParams["sensorParams"]["encoders"]
spParams = modelParams["spParams"]
tmParams = modelParams["tmParams"]


#encodes time of day
#timeOfDayEncoder = DateEncoder(
#timeOfDay=enParams["timestamp_timeOfDay"]["timeOfDay"])
#encodes weekend
#weekendEncoder = DateEncoder(
#weekend=enParams["timestamp_weekend"]["weekend"])
#scalar encoder for chem1
scalarEncoder = RandomDistributedScalarEncoder(
enParams["chem1"]['resolution'])
#scalar encoder for chem2
scalarEncoder = RandomDistributedScalarEncoder(
enParams["chem2"]['resolution'])
#scalar encoder for chem3
scalarEncoder = RandomDistributedScalarEncoder(
enParams["chem3"]['resolution'])
#scalar encoder for chem4
scalarEncoder = RandomDistributedScalarEncoder(
enParams["chem4"]['resolution'])
#encodes scent as a category
categoryEncoder = CategoryEncoder(w=enParams['scent']['w'],categoryList=enParams['scent']['categoryList'],\
                                      forced=enParams['scent']['forced'])



In [5]:
#encode width for full input encoding 

scent_width = 50*scalarEncoder.getWidth()
    
encodingWidth = (scent_width
                 + categoryEncoder.getWidth())

In [6]:
sp = SpatialPooler(
  # How large the input encoding will be.
  inputDimensions=(encodingWidth),
  # How many mini-columns will be in the Spatial Pooler.
  columnDimensions=(2025),
  # What percent of the columns's receptive field is available for potential
  # synapses?
  potentialPct=0.85,
  # This means that the input space has no topology.
  globalInhibition=True,
  localAreaDensity=-1.0,
  # Roughly 2%, giving that there is only one inhibition area because we have
  # turned on globalInhibition (40 / 2048 = 0.0195)
  numActiveColumnsPerInhArea=40.0,
  # How quickly synapses grow and degrade.
  synPermInactiveDec=0.005,
  synPermActiveInc=0.04,
  synPermConnected=0.1,
  # boostStrength controls the strength of boosting. Boosting encourages
  # efficient usage of SP columns.
  boostStrength=3.0,
  # Random number generator seed.
  seed=1956,
  # Determines if inputs at the beginning and end of an input dimension should
  # be considered neighbors when mapping columns to inputs.
  wrapAround=False
)

In [7]:
ArrayOfColumns = numpy.zeros((2025), dtype="int")
activeDutyCycles = numpy.zeros((2025), dtype="float")


#create SDR classifier instance
classifier = SDRClassifierFactory.create(steps=[0])


In [8]:
plt.rcParams['figure.figsize'] = (10.0, 8.0)

In [9]:

lemon = []
flower = []
dog = []
sugar = []
frog = []
ladyfly = []
shock = []
apple = []
cat = []


In [13]:
_INPUT_FILE_PATH = "data50_with_dopa.csv"
numRecords = 100
with open(_INPUT_FILE_PATH, "r") as fin:
    reader = csv.reader(fin)
    headers = reader.next()
    reader.next()
    reader.next()

In [15]:



for count, record in enumerate(reader):
        
    
    if count >= numRecords: break
            

    # Convert data string into Python date object.
    #dateString = datetime.datetime.strptime(record[1], "%m/%d/%y %H:%M")
    scentString = record[0]
    # Convert data value string into float.
        scent = categoryEncoder.encode(record[0])
        chem1 = float(record[1])
        chem2 = float(record[2])
        chem3 = float(record[3])
        chem4 = float(record[4])
        chem5 = float(record[5])
        chem6 = float(record[6])
        chem7 = float(record[7])
        chem8 = float(record[8])
        chem9 = float(record[9])
        chem10 = float(record[10])
        chem11 = float(record[11])
        chem12 = float(record[12])
        chem13 = float(record[13])
        chem14 = float(record[14])
        chem15 = float(record[15])
        chem16 = float(record[16])
        chem17 = float(record[17])
        chem18 = float(record[18])
        chem19 = float(record[19])
        chem20 = float(record[20])
        chem21 = float(record[21])
        chem22 = float(record[22])
        chem23 = float(record[23])
        chem24 = float(record[24])
        chem25 = float(record[25])
        chem26 = float(record[26])
        chem27 = float(record[27])
        chem28 = float(record[28])
        chem29 = float(record[29])
        chem30 = float(record[30])
        chem31 = float(record[31])
        chem32 = float(record[32])
        chem33 = float(record[33])
        chem34 = float(record[34])
        chem35 = float(record[35])
        chem36 = float(record[36])
        chem37 = float(record[37])
        chem38 = float(record[38])
        chem39 = float(record[39])
        chem40 = float(record[40])
        chem41 = float(record[41])
        chem42 = float(record[42])
        chem43 = float(record[43])
        chem44 = float(record[44])
        chem45 = float(record[45])
        chem46 = float(record[46])
        chem47 = float(record[47])
        chem48 = float(record[48])
        chem49 = float(record[49])
        chem50 = float(record[50])
        dopa = int(record[51])
        

        # To encode, we need to provide zero-filled numpy arrays for the encoders
        # to populate.
        #timeOfDayBits = numpy.zeros(timeOfDayEncoder.getWidth())
        #weekendBits = numpy.zeros(weekendEncoder.getWidth())
        scentBits = numpy.zeros(categoryEncoder.getWidth())
        chem1Bits = numpy.zeros(scalarEncoder.getWidth())
        chem2Bits = numpy.zeros(scalarEncoder.getWidth())
        chem3Bits = numpy.zeros(scalarEncoder.getWidth())
        chem4Bits = numpy.zeros(scalarEncoder.getWidth())
        chem5Bits = numpy.zeros(scalarEncoder.getWidth())
        chem6Bits = numpy.zeros(scalarEncoder.getWidth())
        chem7Bits = numpy.zeros(scalarEncoder.getWidth())
        chem8Bits = numpy.zeros(scalarEncoder.getWidth())
        chem9Bits = numpy.zeros(scalarEncoder.getWidth())
        chem10Bits = numpy.zeros(scalarEncoder.getWidth())
        chem11Bits = numpy.zeros(scalarEncoder.getWidth())
        chem12Bits = numpy.zeros(scalarEncoder.getWidth())
        chem13Bits = numpy.zeros(scalarEncoder.getWidth())
        chem14Bits = numpy.zeros(scalarEncoder.getWidth())
        chem15Bits = numpy.zeros(scalarEncoder.getWidth())
        chem16Bits = numpy.zeros(scalarEncoder.getWidth())
        chem17Bits = numpy.zeros(scalarEncoder.getWidth())
        chem18Bits = numpy.zeros(scalarEncoder.getWidth())
        chem19Bits = numpy.zeros(scalarEncoder.getWidth())
        chem20Bits = numpy.zeros(scalarEncoder.getWidth())
        chem21Bits = numpy.zeros(scalarEncoder.getWidth())
        chem22Bits = numpy.zeros(scalarEncoder.getWidth())
        chem23Bits = numpy.zeros(scalarEncoder.getWidth())
        chem24Bits = numpy.zeros(scalarEncoder.getWidth())
        chem25Bits = numpy.zeros(scalarEncoder.getWidth())
        chem26Bits = numpy.zeros(scalarEncoder.getWidth())
        chem27Bits = numpy.zeros(scalarEncoder.getWidth())
        chem28Bits = numpy.zeros(scalarEncoder.getWidth())
        chem29Bits = numpy.zeros(scalarEncoder.getWidth())
        chem30Bits = numpy.zeros(scalarEncoder.getWidth())
        chem31Bits = numpy.zeros(scalarEncoder.getWidth())
        chem32Bits = numpy.zeros(scalarEncoder.getWidth())
        chem33Bits = numpy.zeros(scalarEncoder.getWidth())
        chem34Bits = numpy.zeros(scalarEncoder.getWidth())
        chem35Bits = numpy.zeros(scalarEncoder.getWidth())
        chem36Bits = numpy.zeros(scalarEncoder.getWidth())
        chem37Bits = numpy.zeros(scalarEncoder.getWidth())
        chem38Bits = numpy.zeros(scalarEncoder.getWidth())
        chem39Bits = numpy.zeros(scalarEncoder.getWidth())
        chem40Bits = numpy.zeros(scalarEncoder.getWidth())
        chem41Bits = numpy.zeros(scalarEncoder.getWidth())
        chem42Bits = numpy.zeros(scalarEncoder.getWidth())
        chem43Bits = numpy.zeros(scalarEncoder.getWidth())
        chem44Bits = numpy.zeros(scalarEncoder.getWidth())
        chem45Bits = numpy.zeros(scalarEncoder.getWidth())
        chem46Bits = numpy.zeros(scalarEncoder.getWidth())
        chem47Bits = numpy.zeros(scalarEncoder.getWidth())
        chem48Bits = numpy.zeros(scalarEncoder.getWidth())
        chem49Bits = numpy.zeros(scalarEncoder.getWidth())
        chem50Bits = numpy.zeros(scalarEncoder.getWidth())
        
        
        
        
            

        # Now we call the encoders to create bit representations for each value.
        #timeOfDayEncoder.encodeIntoArray(dateString, timeOfDayBits)
        #weekendEncoder.encodeIntoArray(dateString, weekendBits)
        categoryEncoder.encodeIntoArray(scentString,scentBits)
        scalarEncoder.encodeIntoArray(chem1, chem1Bits)
        scalarEncoder.encodeIntoArray(chem2, chem2Bits)
        scalarEncoder.encodeIntoArray(chem3, chem3Bits)
        scalarEncoder.encodeIntoArray(chem4, chem4Bits)
        scalarEncoder.encodeIntoArray(chem5, chem5Bits)
        scalarEncoder.encodeIntoArray(chem6, chem6Bits)
        scalarEncoder.encodeIntoArray(chem7, chem7Bits)
        scalarEncoder.encodeIntoArray(chem8, chem8Bits)
        scalarEncoder.encodeIntoArray(chem9, chem9Bits)
        scalarEncoder.encodeIntoArray(chem10, chem10Bits)
        scalarEncoder.encodeIntoArray(chem11, chem11Bits)
        scalarEncoder.encodeIntoArray(chem12, chem12Bits)
        scalarEncoder.encodeIntoArray(chem13, chem13Bits)
        scalarEncoder.encodeIntoArray(chem14, chem14Bits)
        scalarEncoder.encodeIntoArray(chem15, chem15Bits)
        scalarEncoder.encodeIntoArray(chem16, chem16Bits)
        scalarEncoder.encodeIntoArray(chem17, chem17Bits)
        scalarEncoder.encodeIntoArray(chem18, chem18Bits)
        scalarEncoder.encodeIntoArray(chem19, chem19Bits)
        scalarEncoder.encodeIntoArray(chem20, chem20Bits)
        scalarEncoder.encodeIntoArray(chem21, chem21Bits)
        scalarEncoder.encodeIntoArray(chem22, chem22Bits)
        scalarEncoder.encodeIntoArray(chem23, chem23Bits)
        scalarEncoder.encodeIntoArray(chem24, chem24Bits)
        scalarEncoder.encodeIntoArray(chem25, chem25Bits)
        scalarEncoder.encodeIntoArray(chem26, chem26Bits)
        scalarEncoder.encodeIntoArray(chem27, chem27Bits)
        scalarEncoder.encodeIntoArray(chem28, chem28Bits)
        scalarEncoder.encodeIntoArray(chem29, chem29Bits)
        scalarEncoder.encodeIntoArray(chem30, chem30Bits)
        scalarEncoder.encodeIntoArray(chem31, chem31Bits)
        scalarEncoder.encodeIntoArray(chem32, chem32Bits)
        scalarEncoder.encodeIntoArray(chem33, chem33Bits)
        scalarEncoder.encodeIntoArray(chem34, chem34Bits)
        scalarEncoder.encodeIntoArray(chem35, chem35Bits)
        scalarEncoder.encodeIntoArray(chem36, chem36Bits)
        scalarEncoder.encodeIntoArray(chem37, chem37Bits)
        scalarEncoder.encodeIntoArray(chem38, chem38Bits)
        scalarEncoder.encodeIntoArray(chem39, chem39Bits)
        scalarEncoder.encodeIntoArray(chem40, chem40Bits)
        scalarEncoder.encodeIntoArray(chem41, chem41Bits)
        scalarEncoder.encodeIntoArray(chem42, chem42Bits)
        scalarEncoder.encodeIntoArray(chem43, chem43Bits)
        scalarEncoder.encodeIntoArray(chem44, chem44Bits)
        scalarEncoder.encodeIntoArray(chem45, chem45Bits)
        scalarEncoder.encodeIntoArray(chem46, chem46Bits)
        scalarEncoder.encodeIntoArray(chem47, chem47Bits)
        scalarEncoder.encodeIntoArray(chem48, chem48Bits)
        scalarEncoder.encodeIntoArray(chem49, chem49Bits)
        scalarEncoder.encodeIntoArray(chem50, chem50Bits)
        
        
        
       

        # Concatenate all these encodings into one large encoding for Spatial
        # Pooling.
            
        encoding = numpy.concatenate(
        [scentBits, chem1Bits, chem2Bits, chem3Bits, chem4Bits,\
        chem5Bits, chem6Bits, chem7Bits, chem8Bits, chem9Bits,\
        chem10Bits, chem11Bits, chem12Bits, chem13Bits, chem14Bits, chem15Bits, chem16Bits, chem17Bits,\
        chem18Bits, chem19Bits, chem20Bits, chem21Bits, chem22Bits, chem23Bits, chem24Bits, chem25Bits,\
        chem26Bits, chem27Bits, chem28Bits, chem29Bits, chem30Bits, chem31Bits, chem32Bits, chem33Bits,\
        chem34Bits, chem35Bits, chem36Bits, chem37Bits, chem38Bits, chem39Bits, chem40Bits, chem41Bits,\
        chem42Bits, chem43Bits, chem44Bits, chem45Bits, chem46Bits, chem47Bits, chem48Bits, chem49Bits, chem50Bits]
        )
        numpy.set_printoptions(threshold=numpy.inf)
        
      
        
        
        # Get the bucket info for this input value for classification.
        bucketIdx = categoryEncoder.getBucketIndices(scentString)[0]
        #use dopa value to learn scent n times in a row before learning next scent
        
        for i in range(dopa):
            
            
            sp.compute(encoding, learn=True, activeArray=ArrayOfColumns)
            
            #active column indices 
        
            sp.getActiveDutyCycles(activeDutyCycles)
        
       
        
        
            activeColumnIndices = numpy.nonzero(ArrayOfColumns)[0]
       
            
            #Heatmap of columns 
            if scentString == ladyfly:
                
                array = numpy.reshape(activeDutyCycles, (45,45))
                norm = plt.colors.Normalize(vmin = 0, vmax = 100, clip = False)
        
        
                plt.pyplot.imshow(array, cmap='hot', interpolation='nearest')
                
    
                plt.pyplot.show()
        
        

        
        # Run classifier to translate active cells back to scalar value.
            classifierResult = classifier.compute(
          recordNum=count,
          patternNZ=activeColumnIndices,
          classification={
        "bucketIdx": bucketIdx,
        "actValue": record[0]
          },
          learn=True,
          infer=True
            )

        # Print the best prediction for 0 steps out.
            probability, value = sorted(
          zip(classifierResult[0], classifierResult["actualValues"]),
        reverse=True
        )[0]
        
        
        
        #print("1-step: {:16} ({:4.4}%)".format(value, probability * 100))
        #print "Actual Value: {} Prediction of: {} has probability of {}.".format(record[0], value, probability*100.0)
        
        
      
        

IndentationError: unexpected indent (<ipython-input-15-e91e1a514757>, line 20)